In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from core import pds as pds
import numpy as np
from datetime import timedelta

In [3]:
# pds frame definition
psd1d_frame = pds.PDS1D_SingleSource()

In [4]:
# setup mesh
nx = 100
dx = 1
x = np.arange(0, nx * dx, dx)

d = 100
u_init = np.zeros(nx)
t0 = 0
lbc = 'Neumann'
rbc = 'Neumann'

In [5]:
# Load test data
from DSS_analyzer_Mariner import Data1D_GAUGE
test_gauge_dataframe = Data1D_GAUGE.Data1D_GAUGE('testdata/pressure_gauge_data.npz')

from DSS_analyzer_Mariner import Data1D_PumpingCurve
stage4_pumping_curve_dataframe = Data1D_PumpingCurve.Data1D_PumpingCurve('testdata/pumping_curve_stage4.npz')
stage5_pumping_curve_dataframe = Data1D_PumpingCurve.Data1D_PumpingCurve('testdata/pumping_curve_stage5.npz')

In [6]:
# Crop the gauge data into three parts
start_time_stg4 = stage4_pumping_curve_dataframe.get_start_time()
end_time_stg4 = stage4_pumping_curve_dataframe.get_end_time()

start_time_stg5 = stage5_pumping_curve_dataframe.get_start_time()
end_time_stg5 = stage5_pumping_curve_dataframe.get_end_time()

In [7]:
stg4_pg_dataframe = test_gauge_dataframe.copy()
stg4_pg_dataframe.crop(start_time_stg4, end_time_stg4)
interval_pg_dataframe = test_gauge_dataframe.copy()
interval_pg_dataframe.crop(end_time_stg4, start_time_stg5)
stg5_pg_dataframe = test_gauge_dataframe.copy()
stg5_pg_dataframe.crop(start_time_stg5, end_time_stg5)
# pass the parameters
psd1d_frame.set_mesh(x)
psd1d_frame.set_diffusivity(d)
psd1d_frame.set_initial(u_init)
psd1d_frame.set_bcs(lbc, rbc)
psd1d_frame.set_t0(t0)
psd1d_frame.set_sourceidx(int(len(x) / 2))  # set the source index at the middle of the mesh
# For the test purpose, crop the gauge data
test_gauge_dataframe = stg4_pg_dataframe.copy()
# decide the time range
test_start_time = start_time_stg4 + timedelta(seconds=200)
test_end_time = test_start_time + timedelta(seconds=200)
test_gauge_dataframe.crop(test_start_time, test_end_time)
# Pre process the PG data
test_gauge_dataframe.data -= np.min(test_gauge_dataframe.data)
# pass the gauge data
psd1d_frame.set_source(test_gauge_dataframe)

Diffusivity is a single scalar value, broadcasted to the mesh length.
Message from pds: Source set done.
Also, just a reminder: please make sure the data is cropped properly.
